# Hw2.2 — Аналіз файлу 2017_jun_final.csv

Обробка локального `2017_jun_final.csv`.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

df = pd.read_csv('2017_jun_final.csv')
print('Перші рядки:')
print(df.head())
print('\nРозмір:', df.shape)
print('\nТипи:')
print(df.dtypes)

# Частка пропусків
print('\nЧастка пропусків:')
print(df.isnull().sum() / len(df))

# Видалимо усі стовпці з пропусками, крім 'Мова програмування'
lang_col = 'Мова програмування'
cols_with_nulls = [c for c in df.columns if df[c].isnull().any() and c != lang_col]
print('\nКолонки з пропусками (які будуть видалені):', cols_with_nulls)
df2 = df.drop(columns=cols_with_nulls)
print('\nПісля видалення колонок — частка пропусків:')
print(df2.isnull().sum() / len(df2))

# Видалимо всі рядки з пропусками у вихідній таблиці
df_dropna = df.dropna()
print('\nРозмір після dropna():', df_dropna.shape)

# Створимо python_data — рядки де вказано Python
python_data = df[df['Мова програмування'].astype(str).str.contains('Python', case=False, na=False)].copy()
print('\nРозмір python_data:', python_data.shape)

# Групування по 'Посада' та агрегація мін/макс зарплати
job_col_candidates = [c for c in df.columns if 'Посада' in c or 'pos' in c.lower()]
salary_candidates = [c for c in df.columns if 'Зарплата' in c or 'salary' in c.lower()]
job_col = job_col_candidates[0] if job_col_candidates else 'Посада'
sal_col = salary_candidates[0] if salary_candidates else 'Зарплата.в.місяць'

agg_minmax = df.groupby(job_col)[sal_col].agg(['min','max'])
print('\nАгрегація min/max по посадах:')
print(agg_minmax.head())

# Функція fill_avg_salary та додавання колонки avg (середня зарплата по посаді)
def fill_avg_salary(group):
    vals = pd.to_numeric(group[sal_col], errors='coerce')
    return vals.mean()

avg_by_job = df.groupby(job_col)[sal_col].apply(lambda s: pd.to_numeric(s, errors='coerce').mean())
df['avg'] = df[job_col].map(avg_by_job)
print('\nДодав стовпець avg — описова статистика:')
print(df['avg'].describe())

# Збереження
df.to_csv('hw2_2_processed.csv', index=False)
print('\nЗбережено hw2_2_processed.csv')
